In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
from pprint import pformat

from hloc import extract_features, match_features, pairs_from_covisibility, pairs_from_retrieval
from hloc import colmap_from_nvm, triangulation, localize_sfm, visualization


In [ ]:
dataset = Path('datasets/aachen/')  # change this if your dataset is somewhere else
images = dataset / 'images/images_upright/'

outputs = Path('outputs/aachen/')  # where everything will be saved
sfm_pairs = outputs / 'pairs-db-covis20.txt'  # top 20 most covisible in SIFT model
loc_pairs = outputs / 'pairs-query-netvlad100.txt'  # top 20 retrieved by NetVLAD
reference_sfm = outputs / 'sfm_superpoint+superglue'  # the SfM model we will build
results = outputs / 'Aachen_hloc_superpoint+superglue_netvlad20.txt'  # the result file

# list the standard configurations available
print(f'Configs for feature extractors:\n{pformat(extract_features.confs)}')
print(f'Configs for feature matchers:\n{pformat(match_features.confs)}')


In [ ]:
retrieval_conf = extract_features.confs['netvlad']
feature_conf = extract_features.confs['superpoint_aachen']
matcher_conf = match_features.confs['superglue']

features = extract_features.main(feature_conf, images, outputs)

In [ ]:
colmap_from_nvm.main(
    dataset / '3D-models/aachen_cvpr2018_db.nvm',
    dataset / '3D-models/database_intrinsics.txt',
    dataset / 'aachen.db',
    outputs / 'sfm_sift')

In [ ]:
pairs_from_covisibility.main(
    outputs / 'sfm_sift', sfm_pairs, num_matched=20)

sfm_matches = match_features.main(matcher_conf, sfm_pairs, feature_conf['output'], outputs)


In [ ]:
reconstruction = triangulation.main(
    reference_sfm,
    outputs / 'sfm_sift',
    images,
    sfm_pairs,
    features,
    sfm_matches)

In [ ]:
global_descriptors = extract_features.main(retrieval_conf, images, outputs)
pairs_from_retrieval.main(global_descriptors, loc_pairs, num_matched=100, db_prefix="db", query_prefix="query")

In [ ]:
### 내가 만든 Globa descriptor par.
loc_pairs = outputs / 'Retrieved_Images.txt'
loc_matches = match_features.main(matcher_conf, loc_pairs, feature_conf['output'], outputs)


In [ ]:
#loc_pairs = outputs / 'Train_PE_officialCrossViT_Tiny_RT_MultiModal_PosEmbedding2_KnowledgeDistillation_lr00001_best_model1243.txt'
results = outputs / 'Pose_Estimation_Results.txt'  # the result file
localize_sfm.main(
    reconstruction,
    dataset / 'queries/*_time_queries_with_intrinsics.txt',
    loc_pairs,
    features,
    loc_matches,
    results,
    covisibility_clustering=False)  # not required with SuperPoint+SuperGlue
